<a href="https://colab.research.google.com/github/rb58853/ML-RSI-Images/blob/main/CLIPEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from PIL import Image
import torch

In [2]:
from transformers import CLIPProcessor, CLIPModel

device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [3]:
def get_image_embedding(image):
    image_process = processor(
       text=None,
       images=image,
       return_tensors="pt"
    )["pixel_values"].to(device)

    embedding = model.get_image_features(image_process)
    # print (embedding)
    # return embedding
    embedding_as_np = embedding.cpu().detach().numpy()
    return embedding
    # return embedding_as_np

In [44]:
def get_text_embedding(text):
    image =  Image.new('RGB', (10, 10), color = (0, 0, 0))
    encoded_text = processor(
        text = [text],
        images = image,
        padding=True,
        # truncation=True,
        # max_length=100,
        return_tensors='pt').to(device)

    outputs = model(**encoded_text)
    text_embeds = outputs['text_embeds']
    text_embedding_as_np = text_embeds.cpu().detach().numpy()
    return text_embeds
    # return text_embedding_as_np

    # pooler_output = outputs['text_model_output']['pooler_output']
    # return pooler_output

In [58]:
from scipy.spatial.distance import cosine

def cosine_similarity(vec1, vec2):
    vec1 = vec1.cpu().detach().numpy()
    vec2 = vec2.cpu().detach().numpy()
    vec1 = vec1[0]
    vec2 = vec2[0]
    return 1 - cosine(vec1, vec2)

In [61]:
image = Image.open('image_4.jpg')
text = 'two dogs in a white and black couch'

image_embedding = get_image_embedding(image)
text_embedding = get_text_embedding(text)

In [62]:
print(cosine_similarity(image_embedding, text_embedding))

0.32101574540138245


In [38]:
def normalize(embeddings):
   norms = embeddings.norm(p=2, dim=-1, keepdim=True)
   return embeddings / norms

def calculate_similarity(text_embedding, image_embedding):
   text_embedding = normalize(text_embedding)
   image_embedding = normalize(image_embedding)
   similarity = torch.matmul(text_embedding, image_embedding.t())
   return similarity


In [39]:
print(calculate_similarity(image_embedding, text_embedding))

tensor([[0.3529]], device='cuda:0', grad_fn=<MmBackward0>)
